# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [1]:
import csv
import cv2
import numpy as np

In [2]:
lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0].split('/')
        try:
            filename = '/'.join(['data',filename[-2],filename[-1]])
        except:
            print(filename, "::", line)
        lines.append([filename, *line[-4:]])

fw = open("./data/driving_log_updated.csv", "w")
for line in lines:
    img = cv2.imread(line[0], cv2.IMREAD_COLOR)
    img = np.fliplr(img)
    new_filename = line[0][:-4]+"_flipped.jpg"
    fw.write("{},{},{},{},{}\n".format(*line))
    fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), line[2], line[3], line[4]))
    cv2.imwrite(new_filename, img)
    
fw.close()

Crop Images

In [3]:
# 55 off top
# 30 off bottom
fw = open("./data/driving_log_updated_cropped.csv", "w")
with open("./data/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0]
        img = cv2.imread(filename)
        img = img[55:-30, :]
        f_split = filename.split("/")
        f_split[-2] = "CROPPED_IMG"
        new_filename = '/'.join(f_split)
        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
        cv2.imwrite(new_filename, img)
    
fw.close()

### Define Datasets

In [4]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.1

train_data = []
test_data = []
validation_data = []

with open("./data/driving_log_updated_cropped.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [5]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


20940 0.7019778746228629
5963 0.19989943010392222
2927 0.09812269527321489


In [6]:
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    samples = sklearn.utils.shuffle(samples)
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size-1]

            vals = np.zeros((batch_size, 4))
            imgs = np.zeros((batch_size, 75, 320, 3))
            index = 0
            for batch_sample in batch_samples:
                filename = batch_sample[0]
                center_image = cv2.imread(filename)
                imgs[index] = center_image
                index += 1
                try:
                    float(batch_sample[1])
                    float(batch_sample[2])
                    float(batch_sample[3])
                    float(batch_sample[4])
                except:
                    print(batch_sample[1], batch_sample[2],batch_sample[3],batch_sample[4])
                vals[index][0] = batch_sample[1]
                vals[index][1] = batch_sample[2]
                vals[index][2] = batch_sample[3]
                vals[index][3] = batch_sample[4]

            # trim image to only see section with road
            X_train = imgs
            y_train = vals
            
            yield X_train, y_train


### Define Hyperparameters

In [12]:
epochs = 50
batch_size = 32

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [13]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (8,8), strides=(4, 4)))
model.add(ELU())
model.add(Conv2D(32, (5,5), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(64))
model.add(ELU())
model.add(Dense(4))


In [15]:
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])

In [16]:
from keras.callbacks import ModelCheckpoint, Callback
checkpoint_callback = ModelCheckpoint('model{epoch:02d}.h5')
model.fit_generator(train_generator, steps_per_epoch=train_data_size/batch_size/2, \
                   validation_data=validation_generator, validation_steps=validation_data_size/batch_size, \
                   epochs=50, max_q_size=1, callbacks=[checkpoint_callback])

Epoch 1/50
328/327 [==============================] - 31s - loss: 21.4508 - acc: 0.9655 - val_loss: 73.9178 - val_acc: 0.9633
Epoch 2/50
328/327 [==============================] - 31s - loss: 18.6630 - acc: 0.9664 - val_loss: 66.0316 - val_acc: 0.9633
Epoch 3/50
328/327 [==============================] - 31s - loss: 18.8378 - acc: 0.9684 - val_loss: 64.2194 - val_acc: 0.9633
Epoch 4/50
328/327 [==============================] - 31s - loss: 18.5014 - acc: 0.9664 - val_loss: 49.2039 - val_acc: 0.9633
Epoch 5/50
328/327 [==============================] - 31s - loss: 49.4391 - acc: 0.9644 - val_loss: 93.9032 - val_acc: 0.9633
Epoch 6/50
328/327 [==============================] - 31s - loss: 18.5041 - acc: 0.9664 - val_loss: 75.9739 - val_acc: 0.9633
Epoch 7/50
328/327 [==============================] - 31s - loss: 18.5976 - acc: 0.9684 - val_loss: 90.1615 - val_acc: 0.9633
Epoch 8/50
328/327 [==============================] - 31s - loss: 18.4112 - acc: 0.9664 - val_loss: 68.9632 - val_acc:

KeyboardInterrupt: 

### Evaluate

In [17]:
metrics = model.evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [18]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 116.35038035183666)
('acc', 0.96490641711229952)


In [ ]:
model.save()